In [1]:
from langchain_groq import ChatGroq # type: ignore

llm = ChatGroq(
    temperature=0,
    groq_api_key = "your_api_key",
    model="llama-3.1-70b-versatile",
)

response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://juspay.io/careers/sde-functional-programming")
page_data = loader.load().pop().page_content
print(page_data)

 Juspay Careers | SDE - Functional Programming

             J U S P A Y       About us    Docs    Integrations      REGION           Southeast Asia (SEA)                Latin America (LATAM)     English  Potugûes              North America                Europe                India              Contact us              SDE - Functional Programming    JUSPAY is a Payments Exchange connecting merchants, payment providers and banks.
With a deep focus on delivering seamless payments UX at scale, we are trusted partners for all major internet companies in India. After surpassing 200M SDK installs and building systems handling the highest payments traffic in India of 2000 TPS, we are now making the next stride to power end-to-end payments for India.
99.999% Availability | 50M Txns/day | 1-Click Experience
What will you be doing?
You’ll be working on the core problems pertaining to our centers of excellence. Each of our Centres of Excellence balances the dual of Framework Development (The Ide

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "SDE - Functional Programming",
  "experience": "more than 1 year of development experience",
  "skills": [
    "Functional Programming (FP)",
    "Backend development",
    "Data Science",
    "Infrastructure development",
    "Haskell/Purescript",
    "Scalable Data Systems"
  ],
  "description": "You’ll be working on the core problems pertaining to our centers of excellence. Each of our Centres of Excellence balances the dual of Framework Development (The Ideal) and Business Enablement (The Practical). You will be working on Backend: Business Logic to handle evolving payment requirements, Data Science: Intelligence from more than a billion transactions to enable automated and smart payment operations, and Infrastructure: Reliable and 99.999% Available infrastructure which can power payments for India."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'SDE - Functional Programming',
 'experience': 'more than 1 year of development experience',
 'skills': ['Functional Programming (FP)',
  'Backend development',
  'Data Science',
  'Infrastructure development',
  'Haskell/Purescript',
  'Scalable Data Systems'],
 'description': 'You’ll be working on the core problems pertaining to our centers of excellence. Each of our Centres of Excellence balances the dual of Framework Development (The Ideal) and Business Enablement (The Practical). You will be working on Backend: Business Logic to handle evolving payment requirements, Data Science: Intelligence from more than a billion transactions to enable automated and smart payment operations, and Infrastructure: Reliable and 99.999% Available infrastructure which can power payments for India.'}

In [15]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
job = json_res
job['skills']

['Functional Programming (FP)',
 'Backend development',
 'Data Science',
 'Infrastructure development',
 'Haskell/Purescript',
 'Scalable Data Systems']

In [21]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [22]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Rohan, a business development manager at Infosys. Infosys is a leading global provider of IT consulting and 
    services, specializing in delivering AI-driven solutions and software that streamline business operations. 
    Over the years, we have helped numerous organizations transform their business through customized digital solutions, 
    enhancing scalability, optimizing processes, reducing costs, and improving overall efficiency.
    Your task is to craft a cold email to the client based on the job description mentioned above, 
    highlighting Infosys's ability to meet their needs.
    Also, include the most relevant case studies or projects from the following links to showcase Infosys's portfolio: {link_list}.
    Write the email as if you are Rohan, BDM at Infosys.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expertise in Functional Programming and Scalable Data Systems for Payment Solutions

Dear Hiring Manager,

I came across your job description for an SDE - Functional Programming role, and I was impressed by the exciting challenges your team is tackling. As a Business Development Manager at Infosys, I'd like to introduce you to our expertise in delivering AI-driven solutions and software that can help streamline your business operations.

Our team has extensive experience in Functional Programming (FP), Backend development, Data Science, and Infrastructure development, aligning perfectly with your requirements. We've worked with clients in the payments industry, helping them build scalable and reliable systems that can handle high volumes of transactions.

I'd like to highlight a few case studies that demonstrate our capabilities:

* We developed a scalable data analytics platform for a leading payment processing company using Haskell and Purescript, which enabled them to proce